In [1]:
import sys
import gc
import math
import numpy as np
import pandas as pd
import optuna
from tqdm import tqdm
import sklearn
import tensorflow as tf
from tensorflow import keras
from typing import Tuple

In [2]:
pd.set_option("use_inf_as_na", True)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
INPUT = '/kaggle/input'
DATA = f'{INPUT}/shopee-product-matching'
OUTPUT = '/kaggle/working'
RESOURCE_DIR = f'{INPUT}/shopee-product-matching-lib/kaggle-shopee-product-matching-1.0'
sys.path.append(f'{INPUT}/sgcharts-ml/src')
sys.path.append(f"{INPUT}/sentence-transformers/sentence-transformers-1.0.4")
sys.path.append(f'{RESOURCE_DIR}/src')
import mylib
import scml
from scml.nlp import strip_punctuation, to_ascii_str
scml.seed_everything()

In [3]:
MODEL = 'efficientnetb0'
CONF = {
    'efficientnetb0': {
        'resolution': 224,
        'output_size': 1280,
    },
    'efficientnetb1': {
        'resolution': 240,
        'output_size': 0,
    },
    'efficientnetb2': {
        'resolution': 260,
        'output_size': 1408,
    },
    'efficientnetb3': {
        'resolution': 300,
        'output_size': 1536,
    },
    'efficientnetb4': {
        'resolution': 380,
        'output_size': 1792,
    },
    'efficientnetb5': {
        'resolution': 456,
        'output_size': 2048,
    },
    'efficientnetb6': {
        'resolution': 528,
        'output_size': 2304,
    },
    'efficientnetb7': {
        'resolution': 600,
        'output_size': 2560,
    },
}
INPUT_SHAPE = (CONF[MODEL]['resolution'], CONF[MODEL]['resolution'], 3)

In [4]:
train = pd.read_csv(f"{DATA}/train.csv", engine="c", low_memory=False)
train["target"] = mylib.target_label(train)
le = sklearn.preprocessing.LabelEncoder()
train["label_group"] = le.fit_transform(train['label_group'])
n_classes=len(le.classes_)
print(f"n_classes={n_classes}")
train.info()

n_classes=11014
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34250 entries, 0 to 34249
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   posting_id   34250 non-null  object
 1   image        34250 non-null  object
 2   image_phash  34250 non-null  object
 3   title        34250 non-null  object
 4   label_group  34250 non-null  int64 
 5   target       34250 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.6+ MB


In [5]:
train.head()

,posting_id,image,image_phash,title,label_group,target
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,666,"[train_129225211, train_2278313361]"
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",7572,"[train_3386243561, train_3423213080]"
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,6172,"[train_2288590299, train_3803689425]"
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi,10509,"[train_2406599165, train_3342059966]"
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,9425,"[train_3369186413, train_921438619]"


In [6]:
def _data_gen(
    dataframe,
    directory,
    target_size,
    batch_size,
    mode,
    color_mode="rgb",
    class_mode="raw",
    x_col="image",
    y_col="label_group"
):
    dtype = np.float32
    rescale = 1./255
    interpolation = "nearest"
    data_format = "channels_last"
    shuffle = False
    idg = keras.preprocessing.image.ImageDataGenerator(
        rescale=rescale,
        data_format=data_format,
        dtype=dtype
    )
    if mode == "train":
        shuffle = True
        idg = keras.preprocessing.image.ImageDataGenerator(
            #shear_range=0.2,
            #zoom_range=0.2,
            #horizontal_flip=True,
            rescale=rescale,
            data_format=data_format,
            dtype=dtype
        )
    g = idg.flow_from_dataframe(
        dataframe=dataframe,
        x_col=x_col,
        y_col=y_col,
        directory=directory,
        target_size=target_size,
        color_mode=color_mode,
        batch_size=batch_size,
        shuffle=shuffle,
        class_mode=class_mode,
        interpolation=interpolation,
    )
    while True:
        x, y = g.next()
        yield [x, y], y

In [7]:
class ArcMarginProduct(keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [8]:
def _model(
    pretrained,
    n_classes: int,
    lr: float,
    input_shape: Tuple[int, int, int],
    dtype=np.float32
):
    #pretrained.trainable = False
    #kernel_initializer = keras.initializers.he_normal()
    #kernel_regularizer = keras.regularizers.l2(0.01)
    image_input = keras.layers.Input(shape=input_shape, name="image_input")
    label_input = keras.layers.Input(shape=(), name="label_input")
    x = pretrained(image_input)
    x = keras.layers.LayerNormalization(name="embedding_output")(x)
    #x = keras.layers.Dense(pretrained.output_shape[1], activation="relu")(x)
    #x = keras.layers.LayerNormalization(name="embedding_output")(x)
    x = ArcMarginProduct(
        n_classes=n_classes, 
        s=30, 
        m=0.5, 
        name='head/arc_margin', 
        dtype=dtype
    )([x, label_input])
    output = tf.keras.layers.Softmax(dtype=dtype)(x)
    model = tf.keras.models.Model(inputs = [image_input, label_input], outputs = [output])
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    loss = keras.losses.SparseCategoricalCrossentropy()
    sca = keras.metrics.SparseCategoricalAccuracy()
    model.compile(loss=loss, optimizer=optimizer, metrics=[sca])
    return model

In [9]:
pretrained = mylib.efficient_net(
    variant=MODEL,
    pooling="avg",
    directory=f"{RESOURCE_DIR}/pretrained/efficientnet",
)
model = _model(
    pretrained=pretrained,
    input_shape=INPUT_SHAPE,
    n_classes=n_classes,
    lr=1e-3,
)
model.summary(line_length=150)

Model: "model"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
image_input (InputLayer)                         [(None, 224, 224, 3)]            0                                                                   
______________________________________________________________________________________________________________________________________________________
efficientnetb0 (Functional)                      (None, 1280)                     4049571           image_input[0][0]                                 
______________________________________________________________________________________________________________________________________________________
embedding_output (LayerNormalization)            (None, 1280)                  

In [10]:
def _callbacks(patience: int, directory: str):
    return [
        keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=patience, verbose=1
        ),
        keras.callbacks.ModelCheckpoint(
            filepath=f"{directory}/model.h5",
            monitor="val_loss",
            save_best_only=True,
            verbose=1
        )
    ]

In [11]:
class MyObjective:
    def __init__(
        self,
        df,
        splitter,
        groups,
        epochs: int,
        batch_size: int,
        patience: int,
        job_dir: str,
        lr: Tuple[float, float],
        n_classes: int,
        input_shape: Tuple[int, int, int],
    ):
        self.df = df
        self.splitter = splitter
        self.epochs = epochs
        self.batch_size = batch_size
        self.patience = patience
        self.job_dir = job_dir
        self.groups = groups
        self.lr = lr
        self.n_classes = n_classes
        self.input_shape = input_shape
        self.history: List[Dict[str, Union[str, int, float]]] = []

    def __call__(self, trial):
        hist = {
            "trial_id": trial.number,
            "learning_rate": trial.suggest_loguniform(
                "learning_rate", self.lr[0], self.lr[1]
            ),
        }
        scores = []
        for fold, (ti, vi) in enumerate(
            self.splitter.split(self.df.index, None, groups=self.groups)
        ):
            train_gen = _data_gen(
                dataframe=self.df.iloc[ti],
                directory=f"{DATA}/train_images",
                target_size=self.input_shape[:2],
                batch_size=self.batch_size,
                mode="train"
            )
            val_gen = _data_gen(
                dataframe=self.df.iloc[vi],
                directory=f"{DATA}/train_images",
                target_size=self.input_shape[:2],
                batch_size=self.batch_size,
                mode="val"
            )
            pretrained = mylib.efficient_net(
                variant=MODEL,
                pooling="avg",
                directory=f"{RESOURCE_DIR}/pretrained/efficientnet",
            )
            model = _model(
                pretrained=pretrained,
                input_shape=self.input_shape,
                n_classes=self.n_classes,
                lr=hist["learning_rate"],
            )
            #model.summary()
            directory = f"{self.job_dir}/trial_{hist['trial_id']}/fold_{fold}"
            history = model.fit(
                train_gen,
                epochs=self.epochs,
                steps_per_epoch=len(ti) / self.batch_size + 1,
                validation_steps=len(vi) / self.batch_size + 1,
                validation_data=val_gen,
                callbacks=_callbacks(self.patience, directory=directory),
                verbose=1
            )
            #y_pred = model.predict(x_val, batch_size=self.batch_size)
            #score = metrics.mean_squared_error(y_val, y_pred, squared=False)
            #print(repr(history.history))
            score = history.history["val_sparse_categorical_accuracy"][-1]
            #log.info(f"score={score:.4f}, fold={fold}, trial={hist['trial_id']}")
            print(f"score={score:.4f}, fold={fold}, trial={hist['trial_id']}")
            hist[f"fold_{fold}_score"] = score
            scores.append(score)
            del model
            gc.collect()
        hist["score_mean"] = np.mean(scores)
        hist["score_std"] = np.std(scores)
        hist["score_worst"] = max(scores)
        self.history.append(hist)
        return hist["score_worst"]

In [12]:
obj = MyObjective(
    df=train,
    splitter=sklearn.model_selection.GroupKFold(n_splits=5),
    groups=train["label_group"],
    epochs=40,
    batch_size=32,  # B3: OOM if batch size > 128 
    patience=6,
    job_dir=OUTPUT,
    lr=(1e-3, 1e-3),
    n_classes=n_classes,
    input_shape=INPUT_SHAPE,
)
study = optuna.create_study(direction="maximize")
study.optimize(obj, n_trials=1)

[I 2021-05-05 05:17:29,264] A new study created in memory with name: no-name-a8f7a48f-dce0-49e7-a333-cadc314a8e3a


Found 27399 validated image filenames.
Epoch 1/40
857/857 [==============================] - 448s 509ms/step - loss: 21.6552 - sparse_categorical_accuracy: 0.0093 - val_loss: 24.7003 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 24.70026, saving model to /kaggle/working/trial_0/fold_0/model.h5
Epoch 2/40
857/857 [==============================] - 347s 405ms/step - loss: 12.0256 - sparse_categorical_accuracy: 0.1451 - val_loss: 28.0634 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 24.70026
Epoch 3/40
857/857 [==============================] - 346s 404ms/step - loss: 6.7674 - sparse_categorical_accuracy: 0.3245 - val_loss: 27.9502 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00003: val_loss did not improve from 24.70026
Epoch 4/40
857/857 [==============================] - 351s 410ms/step - loss: 4.3531 - sparse_categorical_accuracy: 0.4913 - val_loss: 28.7228 - val_sparse_categorical_accuracy

[I 2021-05-05 08:51:34,002] Trial 0 finished with value: 0.0 and parameters: {'learning_rate': 0.001}. Best is trial 0 with value: 0.0.


In [13]:
history = pd.DataFrame.from_records(obj.history)
history.sort_values("score_worst", ascending=False, inplace=True, ignore_index=True)
history.to_csv(f"{OUTPUT}/cv.csv", index=False)
history.head()

,trial_id,learning_rate,fold_0_score,fold_1_score,fold_2_score,fold_3_score,fold_4_score,score_mean,score_std,score_worst
0,0,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
